In [ ]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

In [ ]:
year = 2016
month = 12
tid = '00'
#tid = '18'
station = 'Haukeliseter'
savefig = 0 # 1 = yes, 0 = no

### if plot with double fence data
dofe = 1     # 1 = yes, 0 = no

In [ ]:
#### days 
#t = ['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
t = ['20', '21', '22', '23', '24', '25', '26', '27']
#t = ['22']

In [ ]:
var_name_sfc = 'surface_air_pressure'
var_name_2m = 'air_temperature_2m'
var_name_10m_u = 'x_wind_10m'
var_name_10m_v = 'y_wind_10m'

unit = '[Pa]'
figdir_sfc = '../../Figures/MEPS/%s' %(var_name_sfc)
figdir_2m = '../../Figures/MEPS/%s' %(var_name_2m)
figdir_10m = '../../Figures/MEPS/wind'

# Folder where the figures are being saved    
cF.createFolder('%s/' %(figdir_sfc))
cF.createFolder('%s/' %(figdir_2m))
cF.createFolder('%s/' %(figdir_10m))
form = 'png'

In [ ]:
level = 'sfc'
### MEPS
ncdir_sfc = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)
level = '2m'
ncdir_2m = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)

level = '10m'
ncdir_10m = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)


################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Observations-hour_minutes'

################################################################
### Retrieval 
nc_dir_retrieval = '../../Data/Retrieved_SWC'

################################################################



In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

TA = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
DD   = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
FF   = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

TA0, dt_TA0, hour_TA0, day_TA0, month_TA0, year_TA0 = obsDat.valid_values(TA, DateHour)
dt, Temp, hour, dy, mm, yr = obsDat.arange_daily(TA, DateHour)

   

In [ ]:
idxFF = np.where(FF[:] == -9999)
idxDD = np.where(DD[:] == -9999)

FF[idxFF[0][:]] = np.nan
DD[idxDD[0][:]] = np.nan

### calculate the U, V wind component for barb plot
# http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html

# first calculate the mathematical wind direction in deg
md_deg = 270 - DD
for k in range(0,md_deg.shape[0]):
    if md_deg[k] <0 :
        md_deg[k] = md_deg[k] +360
md_rad = math.pi/180. * md_deg
uwind = FF*np.cos(md_rad)
vwind = FF*np.sin(md_rad)

In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
### Pressure
Haukeli2 = pd.read_csv('%s/%s_pressure.txt' %(txtdir, txt_filename),\
                    sep = ';',header=20, skipfooter = 2, 
                       engine= 'python')


PR = Haukeli2['PR'].astype(float)           # PR;Air pressure at sea level;hPa

PR0, dt_PR0, hour_PR0, day_PR0, month_PR0, year_PR0 = obsDat.valid_values(PR, DateHour)
dt, pres, hour, dy, mm, yr = obsDat.arange_daily(PR, DateHour)

In [ ]:
FF_wind, DD_wind, dt_wind, hour_wind, day_wind, month_wind, year_wind = obsDat.valid_values_wind(FF, DD, DateHour)

In [ ]:
fn_sfc        = dict()
fn_2m         = dict()
fn_10m        = dict()

sfc_pressure = dict()
mslp = dict()
air_temperature_2m = dict()
x_wind_10m    = dict()
y_wind_10m    = dict()

time_sfc      = dict()
time_2m       = dict()
time_10m      = dict()

ini_day       = []
calday        = []
calmon        = []


lead_time_sfc = dict()
lead_time_2m  = dict()
lead_time_10m = dict()

title         = dict()

wd_MEPS    = dict()
ws_MEPS    = dict()

In [ ]:
def plt_variable(lead_time_sfc,wd_MEPS,WD,var):
    
    fig = plt.figure(figsize=(20,7))
    ax = plt.axes()
# Vertical line to show end of day
    ax.axvline(0,color = spagh.vert_col, linewidth = 3)
    ax.axvline(24,color = spagh.vert_col, linewidth = 3)
    ax.axvline(48,color = spagh.vert_col, linewidth = 3)




### ensemble member
    for ens_memb in range(2,10):
        ax.plot(lead_time_sfc[ens_memb],wd_MEPS[ens_memb],color = spagh.memb_col,
           linestyle='-', label='_nolegend_')
    ax.plot(lead_time_sfc[1], wd_MEPS[1], color = spagh.memb_col,
           linestyle = '-', label = 'ensemble member')
    ax.plot(lead_time_sfc[0], wd_MEPS[0], 'k', linewidth = 4, label = 'deterministic')

### observation
    plt.plot(np.arange(0,48), WD,  markersize=20,  linestyle='--', 
         label = 'observation', linewidth= 4)


### fine tuning
    lgd = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.37),
          fancybox=True, shadow=True, ncol=3, fontsize=spagh.label_fs)
    frame = lgd.get_frame()
    frame.set_facecolor('white')

# xaxis
    a = lead_time_sfc[0][0:48]
    ax.set_xlim(-0.5,49-0.5)
    ax.set_xlabel('time', fontsize=spagh.label_fs)
    ax.set_xticks(np.arange(0,49,3))
  #  dates = pvert.dates_plt_00(hour, mm, dy, yr, ini_day)
    xdays = ['%s-%s-%s' %(year,month,ini_day), '',
        6,'',12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+1), '',
        6,'', 12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+2)]
    ax.set_xticklabels(xdays, rotation = 25, fontsize = spagh.tick_fs)
# title
    ax.set_title(title, fontsize=spagh.fontsize, color =spagh.blue )    
    
    if var == 'WD':
        # Horizontal line to show Wind direction
        ax.axhline(90,color=spagh.vert_col, linewidth= 3)
        ax.axhline(180,color=spagh.vert_col, linewidth= 3)
        ax.axhline(270,color=spagh.vert_col, linewidth= 3)
        ax.axhline(360,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylabel('Wind direction', fontsize=spagh.label_fs)
        ax.set_ylim(-0.5,360)
        T = np.arange(0,361,45)
        ax.set_yticks(T)
        ax.set_yticklabels(['N', '', 'E', '','S', '', 'W', '' , 'N'],fontsize = spagh.tick_fs)
    elif var == 'T2':
        ax.axhline(0,color=spagh.vert_col, linewidth= 3)
        # yaxis
        ax.set_ylabel('Air Temperature [$^\circ$C]', fontsize=spagh.label_fs)
        ax.set_ylim(-9,6)
        T = np.arange(-9,7)
        ax.set_yticks(T)
        ax.set_yticklabels([-9, '' , '', -6, '' , '', -3, '' , '', 0, 
                            '' , '', 3, '' , '', 6], fontsize=spagh.tick_fs)
    elif var == 'SP':
        # yaxis
        ax.set_ylabel('Sea Level Pressure [hPa]', fontsize=spagh.label_fs)
        ax.set_ylim(975, 1040,20)
        T = np.arange(970,1040,10)
        ax.set_yticklabels(['' , 980, '', 1000, '', 1020, '', 1040], fontsize=spagh.tick_fs)
        

# tight layout
    plt.tight_layout()
    
    return(lgd)



In [ ]:
#t = ['23']
for day in t:
    ### connect pressure data for two days
    df1 = PR0[int(day)-1]
    df2 = PR0[int(day)]
    Pressure = np.concatenate((df1,df2),axis=0)
        
    ### connect temperature data for two days
    df1 = TA0[int(day)-1]
    df2 = TA0[int(day)]
    Temperature = np.concatenate((df1,df2),axis=0)
    
    ### connect wind data for two days
    df1 = DD_wind[int(day)-1]
    df2 = DD_wind[int(day)]
    WD = np.concatenate((df1,df2),axis=0)
    
    
    for ens_memb in range(0,10):
# read in the netcdf surface file
        fn_sfc[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_sfc,year,month,day,tid,ens_memb))



        time_sfc[ens_memb] = fn_sfc[ens_memb].variables['time']
        time_sfc[ens_memb] = fv.fill_nan(time_sfc[ens_memb][:])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_sfc[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)
# Read in the surface variable (pressure) 
        sfc_pressure[ens_memb] = fn_sfc[ens_memb].variables['surface_air_pressure'][:]
        sfc_pressure[ens_memb] = sfc_pressure[ens_memb]/100
    
        mslp[ens_memb] = fn_sfc[ens_memb].variables['air_pressure_at_sea_level'][:]
        mslp[ens_memb] = mslp[ens_memb]/100
        
# read in the netcdf 2m temperature file
        fn_2m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_2m,year,month,day,tid,ens_memb))
    
        time_2m[ens_memb] = fn_2m[ens_memb].variables['time']
        time_2m[ens_memb] = fv.fill_nan(time_2m[ens_memb][:])
# read in the tempearature variable
        air_temperature_2m[ens_memb] = fn_2m[ens_memb].variables['air_temperature_2m'][:]
        air_temperature_2m[ens_memb] = air_temperature_2m[ens_memb]-273.15

# read in the netcdf 10m wind file
        fn_10m[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_10m,year,month,day,tid,ens_memb))
    
        time_10m[ens_memb] = fn_10m[ens_memb].variables['time']
        time_10m[ens_memb] = fv.fill_nan(time_10m[ens_memb][:])
# read in the wind variable
        x_wind_10m[ens_memb] = fn_10m[ens_memb].variables['x_wind_10m'][:]
        y_wind_10m[ens_memb] = fn_10m[ens_memb].variables['y_wind_10m'][:]
    

# lead time
        lead_time_sfc[ens_memb] = np.arange(0,time_sfc[ens_memb].shape[0])
        lead_time_2m[ens_memb] = np.arange(0,time_2m[ens_memb].shape[0])
        lead_time_10m[ens_memb] = np.arange(0,time_10m[ens_memb].shape[0])

# get only valuable values not nan
        #substitute missing values with nan 
        sfc_pressure[ens_memb] = fv.fill_nan(sfc_pressure[ens_memb][:,:])
        mslp[ens_memb] = fv.fill_nan(mslp[ens_memb][:])
        air_temperature_2m[ens_memb] = fv.fill_nan(air_temperature_2m[ens_memb][:,:])
        x_wind_10m[ens_memb] = fv.fill_nan(x_wind_10m[ens_memb][:,:])
        y_wind_10m[ens_memb] = fv.fill_nan(y_wind_10m[ens_memb][:,:])
#### calculate wind direction and speed
# Create wind speed and direction variables
        wd_MEPS[ens_memb] = np.arctan2(y_wind_10m[ens_memb][:,0], x_wind_10m[ens_memb][:,0])
        ws_MEPS[ens_memb] = np.sqrt((x_wind_10m[ens_memb][:,0])**2 + (y_wind_10m[ens_memb][:,0])**2) 

        wd_MEPS[ens_memb] = 270-np.rad2deg(wd_MEPS[ens_memb])  # convert from math to meteo direction and to degrees
        
        
        larger360 = np.where(wd_MEPS[ens_memb] >360.)
        wd_MEPS[ens_memb][larger360] = wd_MEPS[ens_memb][larger360]-360

        
        lead_time_sfc[ens_memb] = lead_time_sfc[ens_memb][~np.isnan(sfc_pressure[ens_memb][:,0])]
        lead_time_2m[ens_memb] = lead_time_2m[ens_memb][~np.isnan(air_temperature_2m[ens_memb][:,0])]
        lead_time_10m[ens_memb] = lead_time_10m[ens_memb][~np.isnan(wd_MEPS[ens_memb])]
        

        sfc_pressure[ens_memb] = sfc_pressure[ens_memb][~np.isnan(sfc_pressure[ens_memb])]
        mslp[ens_memb] = mslp[ens_memb][~np.isnan(mslp[ens_memb])]
        air_temperature_2m[ens_memb] = air_temperature_2m[ens_memb][~np.isnan(air_temperature_2m[ens_memb])]
#        ws_MEPS[ens_memb] = ws_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]
        wd_MEPS[ens_memb] = wd_MEPS[ens_memb][~np.isnan(wd_MEPS[ens_memb])]    

    if hh < 10:
        hh = '0%s' %hh
    else:
        hh = hh
    
    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
####### plot wind direction     
    lgd = plt_variable(lead_time_10m,wd_MEPS,WD,var='WD')
    
    
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_10m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_10m,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

####### plot Temperature 
    lgd = plt_variable(lead_time_2m,air_temperature_2m,Temperature, var = 'T2')
    
    
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_2m, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_2m,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

    
####### plot Pressure 
    plt_variable(lead_time_sfc,mslp,Pressure, var = 'SP')
    
    
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_sfc, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_sfc,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
    
   
        
    for ens_memb in range(0,10):
        fn_sfc[ens_memb].close()
        fn_2m[ens_memb].close()
        fn_10m[ens_memb].close()